In [1]:
import pandas as pd
import numpy as np


In [2]:
pd.__version__

'1.4.2'

# Biorad 每月報表


## 功能說明：

1. PrintOut_StatReportForExport.xml 匯出成 ALL.XLXS 檔
1. 檢查 QC 結果 `check_qc()`
1. 匯出 IH-500 QC 結果 (排除 QC 失敗) 為 IH500.HTML 及 QC.XLXS 檔
1. 匯出 IH-500 DC-screening II QC 結果 (排除 QC 失敗) 為 DC_SCREENING.HTML 檔
1. 統計全部、每日 LIQ rate 


## 使用方式

1. IH-COM/Report 掛載
1. 至 IH-COM 匯出資料： Print > statistics > ReportForExport
1. 執行本腳本
1. 在印表機資料夾產生三個檔：
    - LIQ： 統計每日 LIQ rate
    - QC： 所有 QC 結果 (排除 QC 失敗)
    - ALL：PrintOut_StatReportForExport.xml 資料
1. 在本腳本之資料夾產生兩個檔：列印以下檔案，組長簽閱後歸檔
    - IH500.HTML: IH-500 QC 結果
    - DC_SCREENING.HTML: IH-500 DC-screening II QC 結果



In [3]:
def check_qc(_series):
    barcode = _series['SampleBarcode']
    anti_A = _series['Anti-A']
    result_text = _series['ResultText']
    dat = _series['DAT poly']
    mono = c3d = _series['C3d']
    igg = _series['IgG']
    screen = _series['I']
    
    # qc1
    if barcode.startswith("0871"):
            if anti_A!="":
                return result_text == "A Rh D negative"
            
            if screen !="":
                return result_text == "ABS positive"
            
            if dat !="":
               return result_text == "DAT negative"
    
    # qc2
    if barcode.startswith("0872"):
            if anti_A!="":
                return result_text == "B Rh D positive"
            
            if screen !="":
                return result_text == "ABS positive"
            
            if dat !="":
                return result_text == "DAT negative"
            
    # qc4
    if barcode.startswith("0874"):            
            if screen !="":
                return result_text == "ABS negative"
            
            if dat !="":
               return result_text == "DAT negative"
    
    # qc7
    if barcode.startswith("0877"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return (result_text == "DAT positive") and (c3d == "-")
            
            if screen !="":
                return result_text == "ABS negative"
            
    # qc8
    if barcode.startswith("0878"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return (result_text == "DAT positive") and (igg == "-")
            
            if screen !="":
                return result_text == "ABS negative"
    
    # lab qc1
    if barcode.startswith("0841"):
        if screen !="":
                return result_text == "ABS negative"
    
    # lab qc2
    if barcode.startswith("0842"):
        if screen !="":
                return result_text == "ABS negative"

            
    
    print(_series)
    return None


In [4]:
def check_liq(series):
    
    for c in series.index:
        if series[c] == 'LIQ':
            return True
        
    return False

In [20]:
try:
    df = pd.read_xml("p:/PrintOut_llStatReportForExport.xml", xpath="//IHComTable")
except:
    df = pd.read_xml("PrintOut_StatReportForExport.xml", xpath="//IHComTable")

    
now_string = pd.Timestamp.now().strftime('%Y%m%d_%H%M')

try:
    df.to_excel(f"F:/Printer/ALL_{now_string}.xlsx")
except OSError:
    df.to_excel(f"ALL_{now_string}.xlsx")

In [21]:
# qc only
qc_df = df[df['SampleBarcode'].str.contains("QC")]
qc_df = qc_df.copy()
qc_df['BGText'] = qc_df['BGText'].fillna("")
qc_df['BGTextABScrDAT'] = qc_df['BGTextABScrDAT'].fillna("")
# remove not interpretable
qc_df = qc_df[~(qc_df['BGText'].str.contains("not"))]
qc_df = qc_df[~(qc_df['BGTextABScrDAT'].str.contains("not"))]

In [22]:
# reform table as TestDate, SampleBarcode, VerifiedByUser, ResultText, [tests....]

group = qc_df.groupby(['SampleBarcode','TestDate'])

sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.FinalResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode','VerifiedByUser', 'ResultText']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    

In [23]:
result_df = pd.concat(sample_df_list)

# remove unverified data
result_df=result_df[~result_df['VerifiedByUser'].isna()]

# fill na with ""
result_df.fillna("", inplace=True)

# remove DP
result_df = result_df.replace("DP", np.nan).dropna()

# remove LIQ
result_df = result_df.replace("LIQ", np.nan).dropna()


result_df.set_index("TestDate", inplace=True)

result_df.sort_index(inplace=True)
result_df.reset_index(inplace=True)

In [24]:


try:
    result_df.to_excel(f"F:/Printer/QC_{now_string}.xlsx")
except OSError:
    result_df.to_excel(f"QC_{now_string}.xlsx")
    

In [27]:
# result_df

In [28]:
start_date = result_df['TestDate'].apply(pd.to_datetime).min().strftime('%Y/%m/%d')
end_date  = result_df['TestDate'].apply(pd.to_datetime).max().strftime('%Y/%m/%d')

title = f"{start_date}-{end_date}"

In [29]:
result_df['QC_check'] = result_df.apply(check_qc, axis=1)
# result_df = result_df.dropna()

TestDate          2025-10-08T16:42:50.651+08:00
SampleBarcode                  08710791251013QC
VerifiedByUser                           034197
ResultText                                     
Anti-A                                         
Anti-B                                         
Anti-D VI-                                     
Ctl                                            
A1                                             
B                                              
1                                            ++
2                                            ++
3                                            ++
4                                             -
5                                             -
6                                             -
7                                             -
8                                            ++
9                                             -
10                                            -
11                                      

In [30]:
result_df[result_df['QC_check'].isnull()]

,TestDate,SampleBarcode,VerifiedByUser,ResultText,Anti-A,Anti-B,Anti-D VI-,Ctl,A1,B,...,9,10,11,I,II,III,DAT poly,IgG,C3d,QC_check
86,2025-10-08T16:42:50.651+08:00,08710791251013QC,034197,,,,,,,,...,-,-,-,,,,,,,None


In [31]:
print("QC failed:")
print(result_df[result_df['QC_check'] == False])

# result_df['QC_check'] = result_df['QC_check'].apply(lambda x: "Failed" if not x else "")
#remove result if qc failed
result_df = result_df[result_df['QC_check']==True].drop("QC_check", axis=1)
result_df2 = result_df[["TestDate","SampleBarcode","VerifiedByUser","ResultText","Anti-A","Anti-B","Anti-D VI-","Ctl","A1","B",
                        "I","II","III", "DAT poly", "IgG", "C3d"]]


QC failed:
Empty DataFrame
Columns: [TestDate, SampleBarcode, VerifiedByUser, ResultText, Anti-A, Anti-B, Anti-D VI-, Ctl, A1, B, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, I, II, III, DAT poly, IgG, C3d, QC_check]
Index: []

[0 rows x 28 columns]


In [32]:
dc_screening_df = result_df[result_df['IgG']!=""][["TestDate","SampleBarcode","VerifiedByUser","ResultText","Ctl", "IgG", "C3d"]]
dc_screening_df

,TestDate,SampleBarcode,VerifiedByUser,ResultText,Ctl,IgG,C3d
8,2025-10-01T14:56:51.382+08:00,08770791251013QC,automatically accepted,DAT positive,-,++,-
9,2025-10-01T14:56:51.697+08:00,08780791251013QC,automatically accepted,DAT positive,-,-,++
44,2025-10-04T13:28:33.799+08:00,08770791251013QC,automatically accepted,DAT positive,-,++,-
45,2025-10-04T13:28:33.993+08:00,08780791251013QC,automatically accepted,DAT positive,-,-,+++
82,2025-10-08T09:33:10.219+08:00,08770791251013QC,automatically accepted,DAT positive,-,++,-
83,2025-10-08T09:33:10.384+08:00,08780791251013QC,automatically accepted,DAT positive,-,-,+++
84,2025-10-08T10:11:45.826+08:00,08770791251013QC,automatically accepted,DAT positive,-,++,-
85,2025-10-08T10:11:45.955+08:00,08780791251013QC,automatically accepted,DAT positive,-,-,++
129,2025-10-13T10:41:39.408+08:00,08770811251110QC,automatically accepted,DAT positive,-,++,-
130,2025-10-13T10:41:39.586+08:00,08780811251110QC,automatically accepted,DAT positive,-,-,+++


In [33]:
result_html = result_df2.to_html(index=False)
dc_html = dc_screening_df.to_html(index=False)

In [49]:



# df['LIQResultText'] = df['InstrumentResultText'].fillna("NT")
# df.loc[df['FinalResultText']=='LIQ', ['LIQResultText']] = "LIQ"

# df['LIQResultText'].value_counts()

In [50]:
# calculate liq rate

#df['OrgResultText'] = df['FinalResultText']
#df.loc[~df['InstrumentResultText'].isnull(),['OrgResultText']] = df[~df['InstrumentResultText'].isnull()]['InstrumentResultText']

df['LIQResultText'] = df['InstrumentResultText'].fillna("NT")
df.loc[df['FinalResultText']=='LIQ', ['LIQResultText']] = "LIQ"
liq_ser = df['LIQResultText'].value_counts()

liq_rate = 0

if 'LIQ' in liq_ser.keys():
    liq_rate = (liq_ser['LIQ']/liq_ser.sum())*100

    


print(f"LIQ RATE: {liq_rate:.2f} %")

liq_ser

LIQ RATE: 0.28 %


NT     13586
?         81
wR        62
LIQ       39
DP        30
+/-        9
E          1
++         1
Name: LIQResultText, dtype: int64

In [51]:
# LIQ rate per day
df['Date2'] = df['TestDate'].apply(pd.to_datetime).apply(lambda x: x.strftime("%Y%m%d"))

liq_count_df = pd.DataFrame(df[['Date2', 'LIQResultText']].value_counts().rename("LIQ")).reset_index()
liq_count_df = liq_count_df[liq_count_df['LIQResultText']=="LIQ"].sort_values("Date2").drop('LIQResultText', axis=1).set_index("Date2")

# Card: Air gap not detected (D22009)
airgap_count_df = pd.DataFrame(df[['Date2', 'ReactionComment']].value_counts().rename("AirGap")).reset_index()
airgap_count_df=airgap_count_df[airgap_count_df['ReactionComment']=="Card: Air gap not detected (D22009)"].drop("ReactionComment", axis=1).sort_values("Date2").set_index("Date2")



total_count_df = pd.DataFrame(df['Date2'].value_counts().rename("total")).sort_index()

liq_df = pd.merge(liq_count_df, airgap_count_df, how="outer", left_index=True, right_index=True).fillna(0)
liq_df = pd.merge(liq_df, total_count_df, how="outer", left_index=True, right_index=True).fillna(0)
liq_df['LIQRate'] = liq_df['LIQ']/liq_df['total'] 
liq_df['GAPRate'] = liq_df['AirGap']/liq_df['total'] 




In [52]:
try:
    liq_df.to_excel(f"F:/Printer/LIQ_{now_string}.xlsx")
except OSError:
    liq_df.to_excel(f"LIQ_{now_string}.xlsx")

In [53]:
group = df.groupby(['SampleBarcode','TestDate'])

sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.LIQResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    
result_df = pd.concat(sample_df_list)

result_df['LIQ'] = result_df.apply(check_liq, axis=1)

liq_rate_sample = 0
if "LIQ" in result_df['LIQ']:
    liq_rate_sample = result_df['LIQ'].value_counts()[True] / result_df['LIQ'].size *100

In [54]:
print(f"LIQ result rate: {liq_rate_sample:.2f} %")

LIQ result rate: 0.00 %


In [22]:
html_string = '''

<!DOCTYPE html>
<html>
<head>
<title>{time_range}</title>

{style}


</head>
<body>

<h1>成大醫院 病理部 血庫組 {title} 品管報表</h1>

<p>
<table width="100%">
	<thead>
		<tr style="text-align:right; font-size:12px; font-family:DFKai-sb;">
			<td>組長:_______________________</td>
		</tr>
	</thead>
	<tbody>
    
        <tr style="font-size:10px; font-family:calibri;">
            <td>{LIQ_rate}</td>
        </tr>
        
        
		<tr>
			<td> 
                
                {result}

            </td>
		</tr>
        
        
	</tbody>
</table>
</p>




</body>
</html>

'''

# print footer
'''
	@media print {
	  footer {
		position: fixed;
		bottom: -15px;
		right: 30px;
		width:100%
		border: 1px solid red;
		font-size: 15px;
		font-family: DFKai-sb;
	  }
       .content-block, p {
		page-break-inside: avoid;
	  }
	}
    
'''
        
style_string='''
<style>
    @media print {
      h1 {page-break-before: always;}
    }
    
    @media screen{
        body {
            width:90%;
            margin: auto auto;
        }
    }

    
	table, th, td{
		border: 0;
	}
    
    h1 {
		text-align:center;
		font-family:DFKai-sb;
		font-size: 20px;
	}
	
	.dataframe {
		width: 100%;
		font-family: calibri;
		font-size: 10px;
		border-bottom: 2px solid black;
		border-top: 2px solid black;
	}
	
	.dataframe th{
		border-bottom: 1px solid black;
		text-align: center;
	}
	
	.dataframe td:nth-child(-n+4){
		text-align: left;
	}
	
	.dataframe td{
		text-align: center;
	}
	
	.dataframe tr:nth-child(even){
		background-color: #f2f2f2;
	}
	
</style>
'''

In [23]:
with open("ih500.html", "w") as f:
     f.write(html_string.format(result=result_html,
                               style=style_string,
                               time_range=title,
                               title="IH-500",
                               LIQ_rate =f"LIQ rate= {liq_rate:.2f} %"))
        
with open("dc_screening.html", "w") as f:
     f.write(html_string.format(result=dc_html,
                               style=style_string,
                               time_range=title,
                               title="DC Screening-II",
                               LIQ_rate=""))
    
